In [1]:
import os
os.chdir("../")


In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weight: str
    params_classes: int
    
        

In [3]:
from src.cnnClassifier.constant import *
from src.cnnClassifier.utils.common import create_directories,read_yaml

d:\End-to-End chicken-diesase-implementation


In [4]:
class ConfigurationManager:
    def __init__(self, configpath = CONFIG_FILE_PATH, paramspath = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(configpath)
        self.params = read_yaml(paramspath)
        
        create_directories([self.config.artifacts_root])
        
    def get_base_model_config(self) -> PrepareBaseModelConfig:
        
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=config.root_dir,
            base_model_path=config.base_model_config,
            updated_base_model=config.updated_base_model_config,
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weight=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
            
        )
        
        return prepare_base_model_config
    

In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
 

In [6]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig) -> None:
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            include_top=self.config.params_include_top,
            weights=self.config.params_weight,
            input_shape=self.config.params_image_size,
            classes=self.config.params_classes,
            
        )
        
        self.save_model(path=self.config.base_model_path, model=self.model)
          
    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainlable = False
                
        elif (freeze_till is not None) and (freeze_till > 0):
            for layers in model.layers[:-freeze_till]:
                model.trainable = False
                
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)
            
            
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )
        
        full_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
                           loss=tf.keras.losses.CategoricalCrossentropy(),
                           metrics=["accuracy"]
                           
                           )
        full_model.summary()
        
        return full_model
        
        
    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        
        self.save_model(path=self.config.updated_base_model, model=self.full_model)
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
            

In [7]:
try:
    config = ConfigurationManager()
    get_base_model_config = config.get_base_model_config()
    prepare_model = PrepareBaseModel(get_base_model_config)
    prepare_model.get_base_model()
    prepare_model.update_base_model()
    
except Exception as e:
    raise(e)
    


[2023-07-30 22:49:37,637 - INFO - common - yaml file: config\config.yaml loaded successfully]
[2023-07-30 22:49:37,649 - INFO - common - yaml file: params.yaml loaded successfully]
[2023-07-30 22:49:37,649 - INFO - common - Directory created ! artifacts]
[2023-07-30 22:49:37,653 - INFO - common - Directory created ! artifacts/prepare_base_model]


[2023-07-30 22:49:38,455 - WARNING - saving_utils - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                       

In [10]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense


In [11]:
model = Sequential()
# model.add(tf.keras.Input(shape=(16,)))
model.add(Dense(30, activation="relu", input_shape=(16,)))
model.add(Dense(20))
model.add(Dense(10))
model.add(Dense(3, activation="softmax"))

model.compile(optimizer="adam", loss="categoricalcross_entropy", metrics=["accuracy"])


In [12]:
model.trainable = False

In [13]:
model.input

<KerasTensor: shape=(None, 16) dtype=float32 (created by layer 'dense_2_input')>

In [28]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 30)                510       
                                                                 
 dense_14 (Dense)            (None, 20)                620       
                                                                 
 dense_15 (Dense)            (None, 10)                210       
                                                                 
 dense_16 (Dense)            (None, 3)                 33        
                                                                 
Total params: 1,373
Trainable params: 0
Non-trainable params: 1,373
_________________________________________________________________
